<a href="https://colab.research.google.com/github/hemantnyadav/Multiple-Time-Series-Forecasting/blob/main/GluonTS_models_on_Retail_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Estimating Product sale as Multiple Time Sereis forecasting using single DeepLearning Model
- Dataset:(811 Product and 52 Weeks)

  It is in total 811 time series with lenght of 52

  Link to dataset [click](https://archive.ics.uci.edu/ml/machine-learning-databases/00396/Sales_Transactions_Dataset_Weekly.csv)
- Models:
  1. DeepAREstimator
  2. TemporalFusionTransformerEstimator
  3. NBEATSEstimator
  4. TransformerEstimator




### Install GluonTS and MXNET

In [17]:
#!pip install --upgrade mxnet==1.6.0
#!pip install gluonts

### Import Required packages

In [271]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
from gluonts.mx import DeepAREstimator,TemporalFusionTransformerEstimator, NBEATSEstimator
from gluonts.mx import TransformerEstimator, WaveNetEstimator
from gluonts.mx import Trainer


### Load Dataset

In [5]:
data  = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00396/Sales_Transactions_Dataset_Weekly.csv")
df_input = data.filter(regex=r'Product|W')
df_input.head()

In [7]:
ts_code = df_input['Product_Code'].astype('category').cat.codes.values

### Train Test Split
Each time series is divided as 41 observation for training and 11 observations for testing

In [10]:
df_train = df_input.iloc[:,1:42].values
df_test = df_input.iloc[:,42:].values
print(df_train.shape, df_test.shape)

### Define Dataset in Gluonts ListData format
In this 
- Set intitial values
- Define list dataset

In [201]:
freq= '1D'
start_train = start = pd.Period("01-01-2019") 
start_test = pd.Timestamp("2014-11-07") 
prediction_length = 6

train_ds = ListDataset(
    [
        {
            FieldName.TARGET: target,
            FieldName.START: start_train,
            FieldName.FEAT_STATIC_CAT: fsc
        }for(target,fsc) in zip(df_train,ts_code.reshape(-1,1))
    ],
    freq = freq
)

test_ds = ListDataset(
    [
        {
            FieldName.TARGET: target,
            FieldName.START: start_test,
            FieldName.FEAT_STATIC_CAT: fsc
        }for(target,fsc) in zip(df_test,ts_code.reshape(-1,1))
    ],
    freq = freq
)

### Evaluate models

In this
- Define Estimators
- Generate predictors
- Evaluate each predictor
- Evaluate using backtest and 
- Store results in dataframe

In [295]:
smape = pd.DataFrame(estimators, index = ['MAPE'])

estimators = {
              'DeepAREstimator':DeepAREstimator,
              'TemporalFusionTransformerEstimator': TemporalFusionTransformerEstimator, 
              'NBEATSEstimator': NBEATSEstimator, 
              'TransformerEstimator': TransformerEstimator
              }

for name,e in estimators.items():
    estimator = e( 
                    freq = "1D",
                    context_length = 6,
                    prediction_length = prediction_length,
                    trainer = Trainer(epochs=3)
                  )
    predictor = estimator.train(training_data = train_ds)
    forecast_it, ts_it = make_evaluation_predictions(
                            dataset=test_ds,
                            predictor = predictor,
                            num_samples=100
                            )
  

  
    tss = list(tqdm(ts_it,total=len(df_test)))
    forecasts = list(tqdm(forecast_it,total=len(df_test)))  

    evaluator = Evaluator(quantiles=[0.1,0.5, 0.9])
    agg_metrics, item_metrics = evaluator(tss, forecasts)
    print(name,":MAPE-" ,agg_metrics['MAPE'])
    smape[name] = agg_metrics['MAPE']

100%|██████████| 50/50 [00:02<00:00, 21.88it/s, epoch=3/3, avg_epoch_loss=2.09]


  0%|          | 0/811 [00:00<?, ?it/s]

  0%|          | 0/811 [00:00<?, ?it/s]

Running evaluation: 811it [00:00, 10169.39it/s]
/usr/local/lib/python3.9/dist-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


DeepAREstimator :MAPE- 0.5636000020088904


...

  0%|          | 0/811 [00:00<?, ?it/s]

  0%|          | 0/811 [00:00<?, ?it/s]

Running evaluation: 811it [00:00, 13787.71it/s]


TransformerEstimator :MAPE- 0.5861525825914958


/usr/local/lib/python3.9/dist-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


### Final Result of Aggregate MAPE for all 811 time series

In [296]:
smape

,DeepAREstimator,TemporalFusionTransformerEstimator,NBEATSEstimator,TransformerEstimator
MAPE,0.5636,0.493085,0.384664,0.586153


# **The End**

---
# Test Experiments...


In [249]:
'''
tft_estimator = TemporalFusionTransformerEstimator( 
                            freq = "1D",
                            context_length = 6,
                            prediction_length = prediction_length,
                            trainer = Trainer(epochs=10)
                            )
predictor = tft_estimator.train(training_data = train_ds)
'''


'\ntft_estimator = TemporalFusionTransformerEstimator( \n                            freq = "1D",\n                            context_length = 6,\n                            prediction_length = prediction_length,\n                            trainer = Trainer(epochs=10)\n                            )\npredictor = tft_estimator.train(training_data = train_ds)\n'

In [251]:

deepar_estimator = DeepAREstimator( freq = "1D",
                            context_length = 6,
                            prediction_length = prediction_length,
                            #use_feat_static_cat = True,
                            #cardinality = [1],
                            #num_layers= 2,
                            #num_cells= 32,
                            #cell_type='lstm',
                            trainer = Trainer(epochs=10)
                            )
predictor = estimator.train(training_data = train_ds)

100%|██████████| 50/50 [00:01<00:00, 26.93it/s, epoch=10/10, avg_epoch_loss=2.03]


In [252]:
from gluonts.evaluation.backtest import make_evaluation_predictions
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds,
    predictor = predictor,
    num_samples=100
)

from tqdm.autonotebook import tqdm
print("Obtaining time Series conditioning values...") 
#forecasts = list(forecast_it)
tss = list(tqdm(ts_it,total=len(df_test)))
print("Obtaining time Series predictions...") 
forecasts = list(tqdm(forecast_it,total=len(df_test)))

Obtaining time Series conditioning values...


  0%|          | 0/811 [00:00<?, ?it/s]

Obtaining time Series predictions...


  0%|          | 0/811 [00:00<?, ?it/s]

In [253]:
def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = prediction_length
    prediction_intervals = (80.0, 95.0)
    legend = ["observations", "median prediction"] + [
        f"{k}% prediction interval" for k in prediction_intervals
    ][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color="g")
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()


In [254]:
df_check = pd.DataFrame(data = tss[0][-prediction_length:])
df_check.rename(columns={0:'Original Time Series(TS_Entry)'}, inplace=True)
df_check['Precticted Values(Forecast Mean)'] = forecasts[0].mean
df_check['Values in Test Dataset'] = df_test[0:1,:][0][-prediction_length:]

In [255]:
df_check

,Original Time Series(TS_Entry),Precticted Values(Forecast Mean),Values in Test Dataset
2014-11-12,12.0,8.706644,12
2014-11-13,3.0,8.733665,3
2014-11-14,7.0,9.337872,7
2014-11-15,6.0,9.381988,6
2014-11-16,5.0,8.236892,5
2014-11-17,10.0,7.802739,10


In [256]:
for i in tqdm(range(6)):
  ts_entry = tss[i]
  forecast_entry = forecasts[i]
  #plot_prob_forecasts(ts_entry, forecast_entry)


  0%|          | 0/6 [00:00<?, ?it/s]

In [257]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.1,0.5, 0.9])
agg_metrics, item_metrics = evaluator(tss, forecasts)

Running evaluation: 811it [00:00, 11121.15it/s]
/usr/local/lib/python3.9/dist-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


In [258]:
item_metrics

,item_id,forecast_start,MSE,abs_error,abs_target_sum,abs_target_mean,seasonal_error,MASE,MAPE,sMAPE,ND,MSIS,QuantileLoss[0.1],Coverage[0.1],QuantileLoss[0.5],Coverage[0.5],QuantileLoss[0.9],Coverage[0.9]
0,None,2014-11-12,12.655004,19.498158,43.0,7.166667,3.25,0.999906,0.636821,0.447039,0.453446,9.030453,7.300472,0.166667,19.498158,0.666667,10.761636,1.000000
1,None,2014-11-12,4.612906,12.760029,17.0,2.833333,2.50,0.850669,0.986849,0.912816,0.750590,4.876257,3.365407,0.000000,12.760028,0.500000,3.762699,0.833333
2,None,2014-11-12,6.576374,9.065941,52.0,8.666667,4.50,0.335776,0.134832,0.159360,0.174345,4.695618,7.470416,0.000000,9.065942,0.166667,6.540610,0.833333
3,None,2014-11-12,15.521028,15.703678,51.0,8.500000,3.00,0.872427,0.265078,0.322712,0.307915,7.509998,9.343458,0.000000,15.703679,0.166667,7.589525,0.833333
4,None,2014-11-12,9.919091,16.837227,45.0,7.500000,3.50,0.801773,0.458603,0.345253,0.374161,7.579120,5.651906,0.000000,16.837228,0.666667,10.564129,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,None,2014-11-12,0.782710,3.106689,3.0,0.500000,0.50,1.035563,0.958771,1.947492,1.035563,25.997464,0.800876,0.000000,3.106689,0.666667,4.446021,0.666667
807,None,2014-11-12,3.931563,10.273279,30.0,5.000000,1.75,0.978408,0.332178,0.406346,0.342443,7.339237,6.034996,0.000000,10.273279,0.000000,2.001305,0.833333
808,None,2014-11-12,3.698318,9.043636,9.0,1.500000,1.25,1.205818,0.848105,1.742662,1.004848,20.107467,1.991714,0.000000,9.043637,0.500000,11.192869,0.500000
809,None,2014-11-12,0.836282,3.052465,3.0,0.500000,0.25,2.034977,0.995064,1.988179,1.017488,68.363286,0.729265,0.000000,3.052465,0.166667,5.115636,0.666667


In [259]:
agg_metrics['MAPE']

0.5834593516635144